In [157]:
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import string
import re
from transformers import logging

logging.set_verbosity_error()

## What is my workflow and what are my tasks?

**Prototype**

Takes a test string as input

Returns whether vegetarian or not vegetarian 


**First iteration**

Takes a list of menu items as input

Returns veg or not veg for all 

EXTRA: can tally or not.

#### We have now narrowed down on the model of our choice for 88% efficiency while using bins = 'vegetarian' and 'meat'

### Create function to do everything

**PROTOTYPE**

In [36]:
def get_diet_proto(test_str):

    # Text classifier model
    classifier = pipeline("zero-shot-classification",
                          model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
    
        
    #classify them
    results = classifier(test_str, ['Vegetarian', 'meat'])
    
    diet = results["labels"][0]
    
    if diet == "meat":
        diet = "Not Vegetarian"
   
    print(test_str)
    print(diet)

In [37]:
test_str1 = "Maitake Mushroom Fritters : Organic Maitake Mushrooms Deep Fried In Crispy Batter"
test_str2 = "Shrimp Cakes : tiger shrimp, atlantic cod + almonds"
test_str3 = "Creekstone Farms Ribeye : sichuan spicy doubanjiang OR cantonese black pepper"
test_str4 = "Eggplant : long pepper + house garlic sauce"
test_str5 = "Spicy Marinated Cucumbers : sesame, garlic + chilli"

In [ ]:
combined_str = "Maitake Mushroom Fritters : Organic Maitake Mushrooms Deep Fried In Crispy Batter\nShrimp Cakes : tiger shrimp, atlantic cod + almonds\nCreekstone Farms Ribeye : sichuan spicy doubanjiang OR cantonese black pepper\nEggplant : long pepper + house garlic sauce\nSpicy Marinated Cucumbers : sesame, garlic + chilli"

In [44]:
get_diet_proto(test_str1)

Maitake Mushroom Fritters : Organic Maitake Mushrooms Deep Fried In Crispy Batter
Vegetarian


In [45]:
get_diet_proto(test_str2)

Shrimp Cakes : tiger shrimp, atlantic cod + almonds
Not Vegetarian


In [46]:
get_diet_proto(test_str3)

Creekstone Farms Ribeye : sichuan spicy doubanjiang OR cantonese black pepper
Not Vegetarian


In [47]:
get_diet_proto(test_str4)

Eggplant : long pepper + house garlic sauce
Vegetarian


In [48]:
get_diet_proto(test_str5)

Spicy Marinated Cucumbers : sesame, garlic + chilli
Vegetarian


In [101]:
def get_diet_first_iter(test_str):
    
    test_list = test_str.split("\n")
    test_list = [item for item in test_list if len(item) > 0]
    
    # Text classifier model
    classifier = pipeline("zero-shot-classification",
                          model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
    

    #classify them
    results = classifier(test_list, ['Vegetarian', 'meat'])
    
    labels = [result["labels"][0] for result in results] 

    return dict(zip(test_list, labels))

In [97]:
combined_str = """Maitake Mushroom Fritters : Organic Maitake Mushrooms Deep Fried In Crispy Batter
Shrimp Cakes : tiger shrimp, atlantic cod + almonds
Creekstone Farms Ribeye : sichuan spicy doubanjiang OR cantonese black pepper
Eggplant : long pepper + house garlic sauce
Spicy Marinated Cucumbers : sesame, garlic + chilli"""

In [103]:
menu_string = """Vermicelli Delight $16 (Korean Sweet Potato Noodles With Assorted Vegetables Marinated With Soy Sauce)
Spicy Korean Cauliflower Bites $16 (With Vegan Mayonnaise Dip)
Spicy Sesame Leaf Tofu Patties $18 (Minced Oyster Mushrooms And Tofu Wrapped In Sesame Leaves)
Stuffed Shiitake Mushrooms $18    
Korean Pancakes Platter $20 (Leek, Kimchi Mushroom & Mung Bean Pancakes With Bean Sprouts And Kabocha Pumpkin)
Organic Wheat Free Kale Pancakes $19 (Made With Sweet Rice Flour)
Plant Based Korean ‘Bulgogi’ Bossam Bites $18(Plant Based ‘Bulgogi’with Pickled Radish And Mushrooms In Spicy Korean Blackraspberry Sauce)"""

In [104]:
get_diet_first_iter(menu_string)

{'Vermicelli Delight $16 (Korean Sweet Potato Noodles With Assorted Vegetables Marinated With Soy Sauce)': 'Vegetarian',
 'Spicy Korean Cauliflower Bites $16 (With Vegan Mayonnaise Dip)': 'Vegetarian',
 'Spicy Sesame Leaf Tofu Patties $18 (Minced Oyster Mushrooms And Tofu Wrapped In Sesame Leaves)': 'Vegetarian',
 'Stuffed Shiitake Mushrooms $18    ': 'Vegetarian',
 'Korean Pancakes Platter $20 (Leek, Kimchi Mushroom & Mung Bean Pancakes With Bean Sprouts And Kabocha Pumpkin)': 'Vegetarian',
 'Organic Wheat Free Kale Pancakes $19 (Made With Sweet Rice Flour)': 'Vegetarian',
 'Plant Based Korean ‘Bulgogi’ Bossam Bites $18(Plant Based ‘Bulgogi’with Pickled Radish And Mushrooms In Spicy Korean Blackraspberry Sauce)': 'Vegetarian'}

In [ ]:
# test_str = """SCALLOP CRUDO
# Pink Apples, Italian Basil, Smoked Pine Nut Brodo
# $21
# TONNATO VITELLO
# Spicy Veal Tartare, Tuna Carpaccio
# $20
# CHRYSANTHEMUM SALAD
# Garlic, Sesame, Parmesan
# $21
# WINTER CHICORIES
# Blood Orange, Roasted Cashews, Black Lime, La Tur
# $20
# BBQ CALAMARI
# Pepperoni Fried Rice, Herbed Labneh
# $19"""

## code for joining together menu items if copy pasted from site

code exists here but not on gradio. Just wrote this extra function in case we ever want ti include better formatting in the event that users literally copy paste menus into the box without formatting. It assumes a lot of formatting and is definitely not perfect

In [152]:
def get_diet_second_iter(test_str):
    
    test_list = test_str.split("\n")
    test_list = [item for item in test_list if len(item) > 0] #removed empty elements
    test_list = [item for item in test_list if not re.search(r"^[\d\$]",item)] #removes prices
    
    # Text classifier model
    classifier = pipeline("zero-shot-classification",
                          model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
    
    test = []
    #classify them
    count = 0
    for i in range(0,len(test_list)//2):
        test.append(test_list[count] + " : " + test_list[count+1])
        count+=2
        
    results = classifier(test, ['Vegetarian', 'meat'])
    
    labels = [result["labels"][0] for result in results] 

    return dict(zip(test, labels))

In [154]:
# test_str = """
# ROASTED CHICKEN
# hen of the woods, scallion
# $34
# HANGER STEAK
# crispy potatoes, salsa bianca
# $34
# IBERIAN DURUC PORK CHOP
# roasted fig & radicchio, apple mostarda, agrodolce
# $62
# PRIME DRY-AGED 20 OZ RIBEYE
# garlic confit, salsa verde
# $95
"""

In [155]:
get_diet_second_iter(test_str)

{'ROASTED CHICKEN : hen of the woods, scallion': 'meat',
 'HANGER STEAK : crispy potatoes, salsa bianca': 'meat',
 'IBERIAN DURUC PORK CHOP : roasted fig & radicchio, apple mostarda, agrodolce': 'meat',
 'PRIME DRY-AGED 20 OZ RIBEYE : garlic confit, salsa verde': 'meat'}